In [9]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

همه = 0
سیاسی = 1
اخبار سلامت = 39
واژه خانه = 43
که بوذ و چه کرد = 44
قرآن = 45
اقتصادی = 4
اجتماعی = 5
ورزشی = 6
سلامت = 25
بین الملل = 3
سیاست خارجی = 2
فرهنگی هنری = 8
محیط زیست = 29
فناوری = 14
علمی = 7
حوادث = 9
سرگرمی = 15
عمومی = 21
کاربران = 22
کافه گفت و گو = 23
سفر و تفریح = 24

In [31]:
def extract_news(start_date = '1401/01/01',end_date = '1401/01/28',subject_id = 0):
    upperframe=[] 
    page_number = 28
    url = 'https://www.asriran.com/fa/archive?service_id=1&sec_id=-1&cat_id='+str(subject_id)+'&rpp=100&from_date='+start_date+'&to_date='+end_date+'&p='
    while True:
        url1 = url + str(page_number)
        page = requests.get(url1)
        soup = BeautifulSoup(page.text, "html.parser")
        articles=soup.find_all('article',attrs={'vizhe_cv col-xs-12 col-sm-6'})
        print(page_number)
        if len(articles)==0 :
            print('finished')
            break
        
        for article in articles:
            title = article.find("a",attrs={'class':"vizhe_title"}).text.strip()
            link = article.find("a").get('href').strip()
            link = 'https://www.asriran.com' + link

            summary = article.find("div",attrs={'class':"vizhe_lead"}).text.strip()
            date = article.find("span",attrs={'class':"tarikh_archive"}).text.strip()

            url2 = link
            page2 = requests.get(url2)
            soup2 = BeautifulSoup(page2.text, "html.parser")

            # IF THE PAGE IS GONE
            ERROR = soup2.find("div",attrs={'class':"error_container"})
            if ERROR :
                subject = 'NOT FOUND'
                body = 'NOT FOUND'
                news_id = 'NOT FOUND'
                short_link = 'NOT FOUND'
                continue # jump to the next article


            subject = soup2.find("div",attrs={'class':"news_path"}).find_all("a")[-1].text

            body = soup2.find("div",attrs={'class':"body"}).text.strip()[:-155]

            news_id = soup2.find("div",attrs={'class':"news_nav news_id_c"}).text.strip()
            news_id = int(re.findall(r'\d+', news_id)[0])

            short_link = soup2.find("div",attrs={'class':"short-link row"}).find("a").get('href').strip()
            short_link = 'https://www.asriran.com' + short_link

            
            frame = [news_id,date,subject,title,summary,body,short_link]
            upperframe.append(frame)

        data=pd.DataFrame(upperframe, columns=['News_ID','Date','Subject','Title','Summary','Body','Short_link'])
        data.to_csv('data'+str(page_number)+'.csv', encoding='utf-8', index=False)
        page_number += 1

    #data=pd.DataFrame(upperframe, columns=['News_ID','Date','Subject','Title','Summary','Body','Short_link'])

    return data

In [32]:
data = extract_news(start_date = '1401/01/01',end_date = '1401/01/28',subject_id = 0)
data.head()

28
29
30
31
32
33
34
35
36
37
38
39
finished


,News_ID,Date,Subject,Title,Summary,Body,Short_link
0,832825,۱۴۰۱/۰۱/۱۰,ورزشی,سخنگوی فدراسیون فوتبال درباره اتفاقات مشهد: نا...,ما به مردم پاسخ داده و عذرخواهی هم کردیم. آن ج...,فارس: سخنگوی فدراسیون فوتبال در مورد اتفاقات ر...,https://www.asriran.com/003Uef
1,832824,۱۴۰۱/۰۱/۱۰,ورزشی,کدام کشورها اجازه ورود زنان به ورزشگاه‌های فوت...,حضور زنان در ورزشگاه‌های الجزایر نیز با محدودی...,اعتماد: تنها در دو کشور ایران و افغانستان زنان...,https://www.asriran.com/003Uee
2,832823,۱۴۰۱/۰۱/۱۰,اقتصادی,درج همزمان قیمت تولید کننده و مصرف کننده,درج همزمان قیمت تولیدکننده و مصرف کننده باید ب...,رئیس سازمان حمایت مصرف‌کنندگان و تولیدکنندگان ...,https://www.asriran.com/003Ued
3,832822,۱۴۰۱/۰۱/۱۰,علمی,هابل دورترین ستاره را رصد کرد,پایگاه خبری اسپیس در این باره نوشت: با ایرندل ...,تلسکوپ فضایی هابل ناسا نور ستاره‌ای را که در ن...,https://www.asriran.com/003Uec
4,832821,۱۴۰۱/۰۱/۱۰,اجتماعی,حضور پلیس در همه پارک‌ها و بوستان‌ها در روز طبیعت,به طور جد و با اقتدار با افرادی که به انحا مخت...,جانشین فرماندهی انتظامی تهران بزرگ از حضور پلی...,https://www.asriran.com/003Ueb


In [ ]:
data.to_csv('data100.csv', encoding='utf-8', index=False)